In [174]:
# # 단어로 나눠서 리스트에 담기 
# news['title'] = news['title'].apply(lambda x : x.split())
# news['title'] = news['title'].apply(lambda x: x+' ')

TypeError: can only concatenate list (not "str") to list

In [191]:
import tensorflow as tf
import re
import pandas
import numpy
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
# from tensorflow.python.keras.preprocessing.sequence import pad_sequences
# from tensorflow.python.keras.preprocessing.text import Tokenizer

In [192]:
# 전처리 
def preprocessing(news, remove_stopwords = False):
    #불용어 제거는 선택이 가능하도록
    
    # HTML 태그 제거
    news_text = BeautifulSoup(news, "html5lib").get_text()
    
    # 영어가 아닌 특수문자를 공백(" ")으로 바꾸기
    news_text = re.sub("[^a-zA-Z]"," ", news_text)
    
    # 대문자를 소문자로 바꾸고 공백 단위로 텍스트를 나누어 리스트화
    words = news_text.lower().split()
    
    # 한글자 없애기
    words = [w for w in words if len(w)>1]
    
    if remove_stopwords:
        # 불용어 제거
        
        # 영어 불용어 사전 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어로 이뤄진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 단어 리스트를 하나의 글로 합성
        clean_news = ' '.join(words)
        
    else: # 불용어 제거 옵션이 안 걸렸을시
        clean_news = ' '.join(words)
        
    clean_list = clean_news.split()
    clean_list.extend(' ')
        
    return clean_list

In [193]:
# csv파일 불러오기 
news = pd.read_csv('../investing_crawl/world-news/news_world-news_00001.csv',index_col=0).dropna()
news

,company,date,title,content
0,By Reuters,"<span>2 minutes ago (Apr 17, 2022 09:25PM ET)<...",U.S. envoy in Seoul for talks over North Korea...,By Josh Smith\nSEOUL (Reuters) - The U.S. envo...
1,By Reuters,"<span>9 minutes ago (Apr 17, 2022 09:18PM ET)<...",Ukrainians defy Russian surrender demand in Ma...,By Oleksandr Kozhukhar and Alessandra Prentice...
2,By Reuters,"<span>37 minutes ago (Apr 17, 2022 08:50PM ET)...",Police hunt for Pittsburgh party shooters who ...,(Reuters) - Police were searching for the shoo...
3,By Reuters,"<span>1 hour ago (Apr 17, 2022 08:21PM ET)</span>",Ukraine completes questionnaire for EU members...,(Reuters) - Ukraine has completed a questionna...
4,By Reuters,"<span>1 hour ago (Apr 17, 2022 08:05PM ET)</span>","Brazil arrests 'Colorido,' PCC drug gang's No....",RIO DE JANEIRO (Reuters) - Brazil's justice mi...
...,...,...,...,...
1713,By Reuters,"<span>Jan 25, 2022 02:06AM ET</span>",U.S. orders departure of Ukraine embassy staff...,By David Shepardson and Costas Pitas\nWASHINGT...
1714,By Reuters,"<span>Jan 25, 2022 12:42AM ET</span>",Burkina Faso army deposes president in West Af...,By Thiam Ndiaga\nOUAGADOUGOU (Reuters) - Burki...
1715,By Reuters,"<span>Jan 25, 2022 12:27AM ET</span>",NASA's new space telescope reaches destination...,By Steve Gorman\n(Reuters) -NASA's James Webb ...
1716,By Reuters,"<span>Jan 24, 2022 11:05PM ET</span>",Tennis-'Where is Peng Shuai?' shirts welcome a...,By Sudipto Ganguly\nMELBOURNE (Reuters) - Fans...


In [194]:
# 함수 적용  
news['clean_title'] = news['title'].apply(preprocessing)
news['clean_content'] = news['content'].apply(preprocessing)

In [195]:
# 날짜 합치고 중복제거 
# 함수 적용  
import re
news['date'] = news['date'].astype(str).str.strip('<span>').str.strip('</span>')
news['date'] = news['date'].str.replace('\n','').str.replace('—','')
news['date'] = news['date'].apply(lambda x: re.findall('\(([^)]+)',x)[0] if re.findall('\(([^)]+)',x) else x )
news['date'] = pd.to_datetime(news['date'].astype(str),utc=True).astype(str).apply(lambda x: x[:10]).astype('datetime64')

C:\Users\bitcamp\anaconda3\envs\cuda\lib\site-packages\dateutil\parser\_parser.py:1212: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [196]:
#date별로 합치기 
news = news.groupby('date')['clean_title','clean_content'].sum()

C:\Users\bitcamp\anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [210]:
# 날짜 정렬
news = news.sort_values(by='date',ascending=True)

In [217]:
# 빈도수 계산 
ser_title = pd.Series(news['clean_title'][0])
ser_title.value_counts()

In [233]:
지수 = pd.read_csv('../석유와가스_지수.csv',index_col=0)
지수= 지수[['Date','전일비']]

In [271]:
지수

,Date,전일비,date
0,2008-01-02,NaN,2008-01-02
1,2008-01-03,-3.766741,2008-01-03
2,2008-01-04,-1.656255,2008-01-04
3,2008-01-07,-3.177211,2008-01-07
4,2008-01-08,3.295744,2008-01-08
...,...,...,...
3530,2022-04-13,2.430165,2022-04-13
3531,2022-04-14,1.103000,2022-04-14
3532,2022-04-15,-0.340995,2022-04-15
3533,2022-04-18,2.586396,2022-04-18


In [ ]:
news.reset_index(inplace=True)

In [283]:

지수['date']=지수['date'].astype('datetime64')

In [307]:
df_merge = pd.merge(지수,news,how='inner')
df_merge.head()

,Date,전일비,date,clean_title,clean_content
0,2022-01-24,-1.557902,2022-01-24,"[tennis, where, is, peng, shuai, shirts, welco...","[by, sudipto, ganguly, melbourne, reuters, fan..."
1,2022-01-25,-4.786643,2022-01-25,"[debt, dispute, probably, sparked, killing, of...","[mexico, city, reuters, the, killing, last, we..."
2,2022-01-26,-0.292913,2022-01-26,"[justice, breyer, fortified, abortion, rights,...","[by, lawrence, hurley, washington, reuters, su..."
3,2022-01-27,-5.978427,2022-01-27,"[search, halted, for, survivors, of, capsized,...","[by, brian, ellsworth, miami, reuters, the, co..."
4,2022-01-28,1.163178,2022-01-28,"[senior, official, to, visit, lithuania, in, s...","[washington, reuters, senior, official, will, ..."


In [289]:
df_merge.drop('Date',axis=1,inplace=True)

In [291]:
df_merge.head()

,전일비,date,clean_title,clean_content
0,-1.557902,2022-01-24,"[tennis, where, is, peng, shuai, shirts, welco...","[by, sudipto, ganguly, melbourne, reuters, fan..."
1,-4.786643,2022-01-25,"[debt, dispute, probably, sparked, killing, of...","[mexico, city, reuters, the, killing, last, we..."
2,-0.292913,2022-01-26,"[justice, breyer, fortified, abortion, rights,...","[by, lawrence, hurley, washington, reuters, su..."
3,-5.978427,2022-01-27,"[search, halted, for, survivors, of, capsized,...","[by, brian, ellsworth, miami, reuters, the, co..."
4,1.163178,2022-01-28,"[senior, official, to, visit, lithuania, in, s...","[washington, reuters, senior, official, will, ..."


In [309]:
# 타이틀만 빼서 보기
df_title = df_merge[['date','전일비','clean_title']]
df_title.head()

,date,전일비,clean_title
0,2022-01-24,-1.557902,"[tennis, where, is, peng, shuai, shirts, welco..."
1,2022-01-25,-4.786643,"[debt, dispute, probably, sparked, killing, of..."
2,2022-01-26,-0.292913,"[justice, breyer, fortified, abortion, rights,..."
3,2022-01-27,-5.978427,"[search, halted, for, survivors, of, capsized,..."
4,2022-01-28,1.163178,"[senior, official, to, visit, lithuania, in, s..."


In [319]:
pd.DataFrame(df_title).to_csv('LDA용_테스트데이터')

In [318]:
df_title

,date,전일비,clean_title
0,2022-01-24,-1.557902,"[tennis, where, is, peng, shuai, shirts, welco..."
1,2022-01-25,-4.786643,"[debt, dispute, probably, sparked, killing, of..."
2,2022-01-26,-0.292913,"[justice, breyer, fortified, abortion, rights,..."
3,2022-01-27,-5.978427,"[search, halted, for, survivors, of, capsized,..."
4,2022-01-28,1.163178,"[senior, official, to, visit, lithuania, in, s..."
5,2022-02-03,1.143301,"[isis, leader, quraishi, kills, himself, durin..."
6,2022-02-04,3.413554,"[want, north, korea, breakthrough, china, tell..."
7,2022-02-07,-0.660128,"[putin, says, some, of, macron, ideas, could, ..."
8,2022-02-08,-4.064312,"[france, macron, calls, for, calm, to, resolve..."
9,2022-02-09,0.650807,"[missouri, texas, to, probe, gofundme, over, r..."
